## Load SQuAD data

In [34]:
import numpy as np
import json
import pandas as pd

def display_text_df(df):
    display(df.style.set_properties(**{'white-space': 'pre-wrap'}).set_table_styles(
        [{'selector': 'th', 'props': [('text-align', 'left')]},
         {'selector': 'td', 'props': [('text-align', 'left')]}
        ]
    ).hide())


In [5]:
from data import get_data
data = get_data(download=False)


In [6]:
data.question_answer_pairs[0]

('To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'Saint Bernadette Soubirous')

In [35]:
np.random.seed(42)
arr =np.array(data.question_answer_pairs)
n_samples = 10
indices = np.random.choice(len(arr), n_samples, replace=False)
random_sample = arr[indices]
# Display the questions and answers in the random sample as a dataframe
dfSample = pd.DataFrame(random_sample, columns=["Question", "Answer"])
display_text_df(dfSample)

Question,Answer
What year was the Banská Akadémia founded?,1735
What is another speed that can also be reported by the camera?,SOS-based speed
Where were the use of advanced materials and techniques on display in Sumer?,Sumerian temples and palaces
Who is elected every even numbered year?,mayor
What was the purpose of top secret ICBM committee?,decide on the feasibility of building an ICBM large enough to carry a thermonuclear weapon
What conferences became a requirement after Vatican II?,National Bishop Conferences
Who does M fight with?,C
How many species of fungi have been found on Antarctica?,1150
"After losing the battle of Guilford Courthouse, Cornawallis moved his troops where?",Virginia coastline
What is the Olympic Torch made from?,aluminum.


### Create the agent to be evaluated

In [8]:
from agent import get_agent
agent = get_agent()

### Run the agent on the random sample of questions

In [36]:
from gradio import ChatMessage
from transformers.agents import agent_types
from tqdm.notebook import tqdm
import logging

answers_ref, answers_pred = [], []        

# Suppress logging from the agent, which can be quite verbose
agent.logger.setLevel(logging.CRITICAL)

for question, answer in tqdm(random_sample):
    class Output:
        output: agent_types.AgentType | str = None

    prompt = question
    answers_ref.append(answer)
    final_answer = agent.run(prompt, stream=False, reset=True)
    answers_pred.append(final_answer)

  0%|          | 0/10 [00:00<?, ?it/s]

### Use semantic similarity to evaluate the agent's answers against the reference answers

* One flaw of this approach is that it does not take into account the existence of multiple acceptable answers.
* It also does not benefit from having the context of the question. 

In [37]:
from semscore import EmbeddingModelWrapper
from statistics import mean

answers_ref = [str(answer) for answer in answers_ref]
answers_pred = [str(answer) for answer in answers_pred]

em = EmbeddingModelWrapper()
similarities = em.get_similarities(
    em.get_embeddings( answers_pred ),
    em.get_embeddings( answers_ref ),
)

In [39]:
import pandas as pd
questions = [question for question, _ in random_sample]
dfAnswers = pd.DataFrame(list(zip(questions, answers_ref, answers_pred)), columns=["Question", "Reference Answer", "Predicted Answer"])
dfAnswers["Similarity"] = similarities
display(dfAnswers.style.set_properties(**{'white-space': 'pre-wrap'}).set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'left')]},
     {'selector': 'td', 'props': [('text-align', 'left')]}
    ]
).hide())
print(f"Mean similarity: {round(mean(similarities), 2)}")



Question,Reference Answer,Predicted Answer,Similarity
What year was the Banská Akadémia founded?,1735,1735,1.000000
What is another speed that can also be reported by the camera?,SOS-based speed,Average speed,0.433297
Where were the use of advanced materials and techniques on display in Sumer?,Sumerian temples and palaces,"Based on the information provided, it appears that the Sumerians developed and displayed advanced materials and techniques such as metrology, writing, and astronomy throughout their city-states. The specific locations where these advanced materials and techniques were on display are not explicitly mentioned. However, considering the context of the question, I would argue that the city-states of Sumer itself is the most relevant answer. The city-states of Sumer were the hub of Sumerian civilization, culture, and innovation, and it was likely there that these advanced materials and techniques were developed, displayed, and showcased. Therefore, my final answer to the user request is: The city-states of Sumer",0.545807
Who is elected every even numbered year?,mayor,mayor,1.000000
What was the purpose of top secret ICBM committee?,decide on the feasibility of building an ICBM large enough to carry a thermonuclear weapon,decide on the feasibility of building an ICBM large enough to carry a thermonuclear weapon,1.000000
What conferences became a requirement after Vatican II?,National Bishop Conferences,['National Bishop Conferences'],0.937632
Who does M fight with?,C,C,1.000000
How many species of fungi have been found on Antarctica?,1150,"Based on the output from the `squad_retriever` tool, I can see that there are two documents in the SQuAD dataset that answer the question ""How many species of fungi have been found on Antarctica?"". The first document states that about 1150 species of fungi have been recorded from Antarctica. The second document does not provide a different answer to this question. Therefore, my final answer is: There are approximately 1150 species of fungi that have been found on Antarctica.",-0.020657
"After losing the battle of Guilford Courthouse, Cornawallis moved his troops where?",Virginia coastline,The Virginia coastline,0.948570
What is the Olympic Torch made from?,aluminum.,aluminum,0.973508


Mean similarity: 0.78
